# Lab: Data-Centric vs Model-Centric approaches

This lab gives an introduction to data-centric vs model-centric approaches to machine learning problems, showing how data-centric approaches can outperform purely model-centric approaches.

In this lab, we'll build a classifier for product reviews (restricted to the magazine category), like:

> Excellent! I look forward to every issue. I had no idea just how much I didn't know.  The letters from the subscribers are educational, too.

Label: ⭐️⭐️⭐️⭐️⭐️ (good)

> My son waited and waited, it took the 6 weeks to get delivered that they said it would but when it got here he was so dissapointed, it only took him a few minutes to read it.

Label: ⭐️ (bad)

We'll work with a dataset that has some issues, and we'll see how we can squeeze only so much performance out of the model by being clever about model choice, searching for better hyperparameters, etc. Then, we'll take a look at the data (as any good data scientist should), develop an understanding of the issues, and use simple approaches to improve the data. Finally, we'll see how improving the data can improve results.

## Installing software

For this lab, you'll need to install [scikit-learn](https://scikit-learn.org/) and [pandas](https://pandas.pydata.org/). If you don't have them installed already, you can install them by running the following cell:

In [ ]:
!pip install scikit-learn pandas

# Loading the data

First, let's load the train/test sets and take a look at the data.

In [1]:
import pandas as pd

/tmp/ipykernel_3905227/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train = pd.read_csv('reviews_train.csv')
test = pd.read_csv('reviews_test.csv')

test.sample(5)

,review,label
37,I love this magazine. Very informative and th...,good
803,not worth the money. Disappointed in reading m...,bad
882,"Awful, mostly ads and not much substance. This...",bad
311,My daughter loves this magazine,good
1,Excellent product! I love reading through the ...,good


In [3]:
train.shape
test.shape

(1000, 2)

# Training a baseline model

There are many approaches for training a sequence classification model for text data. In this lab, we're giving you code that mirrors what you find if you look up [how to train a text classifier](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html), where we'll train an SVM on [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) features (numeric representations of each text field based on word occurrences).

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [5]:
sgd_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [12]:
_ = sgd_clf.fit(train['review'], train['label'])

## Evaluating model accuracy

In [9]:
from sklearn import metrics

In [10]:
def evaluate(clf):
    pred = clf.predict(test['review'])
    acc = metrics.accuracy_score(test['label'], pred)
    print(f'Accuracy: {100*acc:.1f}%')

In [13]:
evaluate(sgd_clf)

Accuracy: 76.8%


## Trying another model

76% accuracy is not great for this binary classification problem. Can you do better with a different model, or by tuning hyperparameters for the SVM trained with SGD?

# Exercise 1

Can you train a more accurate model on the dataset (without changing the dataset)? You might find this [scikit-learn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) handy, as well as the [documentation for supervised learning in scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

One idea for a model you could try is a [naive Bayes classifier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

You could also try experimenting with different values of the model hyperparameters, perhaps tuning them via a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

Or you can even try training multiple different models and [ensembling their predictions](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier), a strategy often used to win prediction competitions like Kaggle.

**Advanced:** If you want to be more ambitious, you could try an even fancier model, like training a Transformer neural network. If you go with that, you'll want to fine-tune a pre-trained model. This [guide from HuggingFace](https://huggingface.co/docs/transformers/training) may be helpful.

### Naive Bayes Classifier


In [7]:
# YOUR CODE HERE
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import FunctionTransformer

to_dense = FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)

# evaluate your model and see if it does better
# than the ones we provided

In [15]:
gnb_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('to_dense', to_dense), # a detail worth to be noticed
    ('clf', GaussianNB()),
])

_ = gnb_clf.fit(train['review'], train['label'])

In [16]:
evaluate(gnb_clf)

Accuracy: 54.1%


Performance of Naive Bayes is poorer than the technical mentioned above. Maybe we need to try various tricks to make it better.

In [20]:
from sklearn.feature_selection import SelectKBest, chi2

In [21]:
gnb_clf_new = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('select', SelectKBest(chi2, k=1000)), 
    ('to_dense', to_dense), # a detail worth to be noticed
    ('clf', GaussianNB()),
])

_ = gnb_clf_new.fit(train['review'], train['label'])

In [22]:
evaluate(gnb_clf_new)

Accuracy: 61.0%


If your features are mainly counts or frequencies, Multinomial NB usually performs better than Gaussian NB.

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
mnb_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

_ = mnb_clf.fit(train['review'], train['label'])

In [25]:
evaluate(mnb_clf)

Accuracy: 85.3%


Wow, an amazing performance. I think I need to know the difference between these two methods. ***To do some survey***

### Hyperparameter Tuning


Using the SGDClassifier to make comparison.

In [26]:
from sklearn.model_selection import GridSearchCV

In [28]:
# YOUR CODE HERE
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

grid_search = GridSearchCV(sgd_clf, parameters, n_jobs=-1)
grid_search.fit(train['review'], train['label'])
# evaluate your model and see if it does better
# than the ones we provided

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 0.001),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2)]})

In [29]:
evaluate(grid_search)

Accuracy: 89.3%


The search space is not large, but it improves the performance a lot, indicating that the method is very sensitive to hyperparameters.

### Ensembling Learning

In [34]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [39]:
# YOUR CODE HERE
clf1 = MultinomialNB()
clf2 = LogisticRegression(max_iter=1000)
clf3 = SVC(probability=True)

eclf = VotingClassifier(estimators=[
        ('nb', clf1), ('lr', clf2), ('svc', clf3)],
        voting='soft',  # 使用软投票
        weights=[1,1,1]  # 可以调整各分类器的权重
    )

esb_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    # ('select', SelectKBest(chi2, k=1000)), # not always make sense, why? It's the same data, only model changes.
    ('eclf', eclf),
])

_ = esb_clf.fit(train['review'], train['label'])
# evaluate your model and see if it does better
# than the ones we provided

In [40]:
evaluate(esb_clf)

Accuracy: 83.6%


Esembling costs more time training but gains less performance, I think there may exists some tricks in selecting classifiers. ***To do some survey***

### Pretrained Transformer

In [1]:
import numpy as np
import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import datasets
from datasets import Dataset, DatasetDict, ClassLabel

/home/xinyuwei/anaconda3/envs/avatar/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Taking a closer look at the training data

Let's actually take a look at some of the training data:

In [9]:
train.head()

,review,label
0,Based on all the negative comments about Taste...,good
1,I still have not received this. Obviously I c...,bad
2,</tr>The magazine is not worth the cost of sub...,good
3,This magazine is basically ads. Kindve worthle...,bad
4,"The only thing I've recieved, so far, is the b...",bad


Zooming in on one particular data point:

In [10]:
print(train.iloc[0].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}


This data point is labeled "good", but it's clearly a negative review. Also, it looks like there's some funny HTML stuff at the end.

# Exercise 2

Take a look at some more examples in the dataset. Do you notice any patterns with bad data points?

In [47]:
# YOUR CODE HERE
train[train['label']=='good']

,review,label
0,Based on all the negative comments about Taste...,good
2,</tr>The magazine is not worth the cost of sub...,good
5,"The magazines are great, but I never received ...",good
6,This is one magazine I really love. It has pri...,good
8,Forever the best magazine! Love it!!,good
...,...,...
6658,Excellent! I look forward to every issue. I ha...,good
6659,I have always loved Readers Digest!,good
6660,Really enjoy te information in this magazine,good
6664,I love this magazine and I am very excited to ...,good


In [48]:
train[train['label']=='good'].iloc[2].to_dict()

{'review': 'The magazines are great, but I never received the golf shoe bag that was supposed to accompany my subscription...<body>',
 'label': 'good'}

## Issues in the data

It looks like there's some funny HTML tags in our dataset, and those datapoints have nonsense labels. Maybe this dataset was collected by scraping the internet, and the HTML wasn't quite parsed correctly in all cases.

# Exercise 3

To address this, a simple approach we might try is to throw out the bad data points, and train our model on only the "clean" data.

Come up with a simple heuristic to identify data points containing HTML, and filter out the bad data points to create a cleaned training set.

In [49]:
import re

In [50]:
def is_bad_data(review: str) -> bool:
    # YOUR CODE HERE
    html_tag_pattern = re.compile('<.*?>')
    
    if re.search(html_tag_pattern, review):
        return True  
    else:
        return False 

## Creating the cleaned training set

In [51]:
train_clean = train[~train['review'].map(is_bad_data)]

## Evaluating a model trained on the clean training set

In [52]:
from sklearn import clone

In [53]:
sgd_clf_clean = clone(sgd_clf)

In [54]:
_ = sgd_clf_clean.fit(train_clean['review'], train_clean['label'])

This model should do significantly better:

In [55]:
evaluate(sgd_clf_clean)

Accuracy: 97.0%
